In [1]:
ORIGIN_DATA = "origin_data"
DATA_CSV = "origin_data_csv"
LOGS = "logs"
MATCH = "Z:"
import pandas as pd

import numpy as np
import wfdb                                      # 讀取 WFDB header / record :contentReference[oaicite:4]{index=4}
from pathlib import Path                         # 物件導向檔案操作 :contentReference[oaicite:5]{index=5}
from datetime import datetime, timedelta, date, time
from tqdm import tqdm                            # 進度列（可省略）
import logging, os                               # 紀錄檔與系統路徑
from collections import defaultdict
import logging
import os
import ast
import re
from typing import List, Optional, Tuple,Set
from  tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

import ast
from typing import List, Union
import pandas as pd
import numpy as np

def _safe_literal(val: str):
    """安全地轉為 Python 物件；失敗就回 None。"""
    try:
        return ast.literal_eval(val)
    except Exception:
        return None

def _parse_icd9(val: str) -> List[int]:
    """將 '[1,2,3]' → [1,2,3]；不合法或空值回 []."""
    if pd.isna(val) or val.strip() in ("", "[]"):
        return []
    items = _safe_literal(val)
    if items is None:
        return []
    return [int(x) for x in items]

def _parse_ecg_list(val: str) -> List[pd.Timestamp]:
    """將 '[2160/7/5 18:27,...]' 轉 List[pd.Timestamp]；空值回 []."""
    if pd.isna(val) or val.strip() in ("", "[]"):
        return []
    # 去掉頭尾中括號再以逗號切
    ts_list = val.strip("[]").split(",")
    return [pd.to_datetime(ts.strip(), errors="coerce") for ts in ts_list]

def _parse_datetime(val: str) -> pd.Timestamp:
    return pd.to_datetime(val, errors="coerce")

def load_patient_csv(path: str) -> pd.DataFrame:
    date_cols = ["ECG_DATETIME", "ADMITTIME", "DISCHTIME", "DEATHTIME", "DOB"]
    converters = {
        **{c: _parse_datetime for c in date_cols},
        "ICD9_CODE": _parse_icd9,
        "ECG_DATETIME_y": _parse_ecg_list,
    }
    df = pd.read_csv(path, converters=converters)

    # age 若為 object → 轉 numeric
    if df["age"].dtype == "O":
        df.loc[:, "age"] = pd.to_numeric(df["age"], errors="coerce")

    return df

modified_alive_427 = load_patient_csv(".\experiment_data_from_yuran\modified_活人_ICD9_427_20250211_with_age+TIME_LENGTH.csv")
print(modified_alive_427.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   SUBJECT_ID      2793 non-null   int64         
 1   ECG_DATETIME    2793 non-null   datetime64[ns]
 2   HADM_ID         2793 non-null   int64         
 3   ICD9_CODE       2793 non-null   object        
 4   ADMITTIME       2793 non-null   datetime64[ns]
 5   DISCHTIME       2793 non-null   datetime64[ns]
 6   DEATHTIME       0 non-null      datetime64[ns]
 7   ECG_DATETIME_y  2793 non-null   object        
 8   GENDER          2793 non-null   object        
 9   DOB             2793 non-null   datetime64[ns]
 10  wave_id         2793 non-null   int64         
 11  age             2793 non-null   int64         
 12  death           2793 non-null   int64         
 13  time_length     2793 non-null   float64       
dtypes: datetime64[ns](5), float64(1), int64(5), object(3)
me

In [8]:
basic_match_ecg_df = pd.read_csv(r".\logs\base_match_ecg_df.csv")
basic_match_ecg_df_drop = basic_match_ecg_df.dropna(subset=['T0'])

merge_df = basic_match_ecg_df_drop.merge(
    modified_alive_427,
    on=['SUBJECT_ID','HADM_ID'],
    how="left"
)

merge_df['diff'] = merge_df['Total_lead2_sec'] - merge_df['time_length']
print(merge_df[merge_df['diff'].abs() > 60]['SUBJECT_ID'])
merge_df.to_csv(os.path.join(LOGS,"test.csv"),index=False)

10        625
17        787
42       1604
87       3768
99       4175
        ...  
2128    95536
2164    97018
2183    98390
2202    99038
2210    99186
Name: SUBJECT_ID, Length: 108, dtype: int64
